In [1]:
import os
os.getcwd()

'/Users/nithinpradeep/AV/Projects/AI-vs-Real-Art-classification-end-to-end/research'

In [2]:
os.chdir('../')
os.getcwd()

'/Users/nithinpradeep/AV/Projects/AI-vs-Real-Art-classification-end-to-end'

In [3]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/nithinpradeep38/AI-vs-Real-Art-classification-end-to-end.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="nithinpradeep38"
os.environ["MLFLOW_TRACKING_PASSWORD"]="3f2a9247def70a4dc84429d810c22229b66065d6"

### Export the following in the terminal to set environment

export MLFLOW_TRACKING_URI=https://dagshub.com/nithinpradeep38/AI-vs-Real-Art-classification-end-to-end.mlflow \
export MLFLOW_TRACKING_USERNAME=nithinpradeep38 \
export MLFLOW_TRACKING_PASSWORD= __password__

In [4]:
import tensorflow as tf

In [5]:
model= tf.keras.models.load_model("/Users/nithinpradeep/AV/Projects/AI-vs-Real-Art-classification-end-to-end/artifacts/training/model.h5")

2024-04-25 23:19:01.790151: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Pro
2024-04-25 23:19:01.790178: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-04-25 23:19:01.790183: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-04-25 23:19:01.790214: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-04-25 23:19:01.790230: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


### Step 1. Create Entity

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

### Step 2: Configuration

In [7]:
from AI_Real_Classifier.constants import *
from AI_Real_Classifier.utils.common import read_yaml, create_directories, save_json
from AI_Real_Classifier.config.configuration import TrainingConfig

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root]) #this will create the artifacts folder

    def get_training_config(self)-> TrainingConfig:
        training= self.config.model_trainer
        prepare_base_model= self.config.prepare_base_model
        params= self.params
        training_data= os.path.join(self.config.data_ingestion.unzip_dir, "dataset")
        class_fake= os.path.join(self.config.data_ingestion.unzip_dir, "dataset", "fakeV2", "fake-v2" )
        class_real= os.path.join(self.config.data_ingestion.unzip_dir, "dataset","real")
        create_directories([Path(training.root_dir)])

        training_config= TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path= Path(training.trained_model_path),
            updated_base_model_path= Path(prepare_base_model.updated_base_model_path),
            training_data= Path(training_data),
            class_fake= Path(class_fake),
            class_real= Path(class_real),
            params_epochs= params.EPOCHS,
            params_batch_size= params.BATCH_SIZE,
            params_is_augmentation= params.AUGMENTATION,
            params_image_size= params.IMAGE_SIZE

            )
        return training_config

    
    def get_evaluation_config(self)-> EvaluationConfig:
        eval_config= EvaluationConfig(
            path_of_model= "artifacts/training/model.h5",
            training_data= "artifats/data_ingestion/dataset",
            mlflow_uri= "https://dagshub.com/nithinpradeep38/AI-vs-Real-Art-classification-end-to-end.mlflow",
            all_params= self.params,
            params_image_size= self.params.IMAGE_SIZE,
            params_batch_size= self.params.BATCH_SIZE

        )
        return eval_config

### Step 3: Component

In [8]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse


In [9]:
class ModelEvaluation:
    def __init__(self,validation_generator, config: EvaluationConfig):
        self.config= config
        self.validation_generator= validation_generator
        

    @staticmethod
    def load_model(path: Path)-> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def save_score(self):
        scores= {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path= Path("scores.json"), data= scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri) #set the registered uri
        tracking_url_type_store= urlparse(mlflow.get_tracking_uri()).scheme #checks if is it file or remote server (http)

        with mlflow.start_run() as run_id:
            mlflow.log_params(self.config.all_params) #first log all the parameters from config
            mlflow.log_metrics(     
                {"loss": self.score[0],
                 "accuracy": self.score[1]}             #log the result from evaluation on validation data 
            )

            #model registry does not work with file store
            if tracking_url_type_store != "file":

                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model") #track in registered uri

            else: 
                mlflow.keras.log_model(self.model, "model") #track in local system

    
    def evaluation(self):
        self.model= self.load_model(self.config.path_of_model)
        self.score= model.evaluate(self.validation_generator)
        self.save_score()

In [10]:
from AI_Real_Classifier.components.model_training import Training
try:
    config= ConfigurationManager()
    training_config= config.get_training_config()
    training= Training(config= training_config)
    training.pre_process()
    training.train_valid_generator()
    eval_config= config.get_evaluation_config()
    evaluation= ModelEvaluation(validation_generator=training.validation_generator, config=eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
    raise e

Found 6224 validated image filenames belonging to 2 classes.
Found 1556 validated image filenames belonging to 2 classes.


/opt/anaconda3/envs/aireal/lib/python3.8/site-packages/PIL/Image.py:1000: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
2024-04-25 23:19:42.876159: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


25/25 [==============================] - 25s 983ms/step - loss: 72.7709 - accuracy: 0.8425


2024/04/25 23:20:09 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /var/folders/7g/0k1jm7qx5h95s6mvj4b1pj300000gn/T/tmpcxf84z96/model/data/model/assets


/opt/anaconda3/envs/aireal/lib/python3.8/site-packages/_distutils_hack/__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2024/04/25 23:21:33 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 2
Created version '2' of model 'VGG16Model'.


In [10]:
print(tf.__version__)

2.13.0


In [11]:
import keras

print(keras.__version__)


2.13.1


In [12]:
import mlflow
print(mlflow.__version__)

2.12.1
